### Extract Required Information and Convert to Appropriate Data Structure

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

digits = "0123456789"


def extract_ints(row):

    current = ""
    ids = []

    for c in row[1]:

        if c in digits:
            current += c
        elif current != "":
            ids.append(int(current))
            # ids.append(current)
            current = ""

    return sorted(ids)


def extract_ids(row):

    current = ""
    ids = []

    for c in row[1]:

        if c in digits:
            current += c
        elif current != "":
            # ids.append(int(current))
            ids.append(current)
            current = ""

    return sorted(ids)


df_pp = pd.read_csv('/content/drive/MyDrive/FoodGPT/Code/data/PP_recipes.csv')
print(df_pp.shape)
df_pp.head()

df_ingr = pd.read_pickle('/content/drive/MyDrive/FoodGPT/Code/data/ingr_map.pkl')
df_ingr.head()

df_raw = pd.read_csv('/content/drive/MyDrive/FoodGPT/Code/data/RAW_recipes.csv')
df_raw.head()

df_raw_trim = df_raw[['name', 'id', 'ingredients']]
df_raw_trim.head()

df_new = pd.merge(df_pp, df_raw_trim, on='id', how='inner')
df_new = df_new.drop(['steps_tokens', 'techniques', 'calorie_level', 'name_tokens', 'ingredient_tokens'], axis=1)
pd.set_option('max_colwidth', None)

name = df_new.pop('name')
df_new.insert(1, 'name', name)

index = df_new.pop('i')
df_new.insert(0, 'index', index)

df_new = df_new.drop(['id'], axis=1)

df_new = df_new.sort_values('index', ignore_index=True)
df_new = df_new.drop(['index'], axis=1)


df_new['ids'] = df_new.apply(extract_ids, axis=1)
df_new['int_ids'] = df_new.apply(extract_ints, axis=1)

df_new.to_pickle('/content/drive/MyDrive/FoodGPT/Code/data/complete_recipe_dataset.pkl')


Mounted at /content/drive
(178265, 8)


In [ ]:
all = pd.read_pickle('/content/drive/MyDrive/FoodGPT/Code/data/complete_recipe_dataset.pkl')
all = all.reset_index()
all["index"] = all
all.head()

,index,name,ingredient_ids,ingredients,ids,int_ids
0,0,white bean green chile pepper soup,"[3384, 7979, 2127, 3502, 3217, 1257, 2778, 5006, 6654]","['great northern beans', 'yellow onion', 'diced green chilies', 'ground cumin', 'garlic powder', 'fat-free chicken broth', 'fresh cilantro leaves', 'extra virgin olive oil', 'sour cream']","[1257, 2127, 2778, 3217, 3384, 3502, 5006, 6654, 7979]","[1257, 2127, 2778, 3217, 3384, 3502, 5006, 6654, 7979]"
1,1,devilicious cookie cake delights,"[912, 7557, 2499, 5382]","[""devil's food cake mix"", 'vegetable oil', 'eggs', ""reese's pieces""]","[2499, 5382, 7557, 912]","[912, 2499, 5382, 7557]"
2,2,baked potato toppings,"[4623, 6265, 1168, 6016, 3597, 3440, 7213, 1699, 5340, 4836, 5180, 5485, 2982]","['mayonnaise', 'salsa', 'cheddar cheese', 'refried beans', 'ham', 'green pepper', 'tomatoes', 'cooked bacon', 'pepperoni', 'mozzarella cheese', 'parmesan cheese', 'pizza sauce', 'frozen chopped broccoli']","[1168, 1699, 2982, 3440, 3597, 4623, 4836, 5180, 5340, 5485, 6016, 6265, 7213]","[1168, 1699, 2982, 3440, 3597, 4623, 4836, 5180, 5340, 5485, 6016, 6265, 7213]"
3,3,kfc honey bbq strips,"[1304, 2683, 3217, 6270, 3532, 869, 7557, 3698, 7655, 3723, 4053, 4436]","['chicken tenders', 'flour', 'garlic powder', 'salt', 'ground pepper', 'buttermilk', 'vegetable oil', 'hickory flavored barbecue sauce', 'water', 'honey', 'ketchup', 'liquid smoke']","[1304, 2683, 3217, 3532, 3698, 3723, 4053, 4436, 6270, 7557, 7655, 869]","[869, 1304, 2683, 3217, 3532, 3698, 3723, 4053, 4436, 6270, 7557, 7655]"
4,4,lamb stew with tomatoes chickpeas and spices,"[4130, 6270, 3486, 7557, 5010, 3203, 2683, 1257, 7213, 393, 3500, 3502, 3497, 3512, 1313, 2856]","['lamb shoulder', 'salt', 'ground black pepper', 'vegetable oil', 'onions', 'garlic cloves', 'flour', 'low sodium chicken broth', 'tomatoes with juice', 'bay leaves', 'ground coriander', 'ground cumin', 'ground cinnamon', 'ground ginger', 'chickpeas', 'fresh parsley']","[1257, 1313, 2683, 2856, 3203, 3486, 3497, 3500, 3502, 3512, 393, 4130, 5010, 6270, 7213, 7557]","[393, 1257, 1313, 2683, 2856, 3203, 3486, 3497, 3500, 3502, 3512, 4130, 5010, 6270, 7213, 7557]"


In [ ]:
from gensim.models import Word2Vec

data_list = df_new["ids"].values.tolist()
model = Word2Vec(data_list, min_count=1, workers=3, window=10, sg=0)

In [ ]:
model.wv.save('/content/drive/MyDrive/FoodGPT/Code/models/ingredient_similarities.model')